In [23]:
import pandas as pd
import sys
import os
from pathlib import Path
import sqlite3

project_root = os.path.abspath('../../')
if project_root not in sys.path:
    sys.path.append(project_root)
    
from src.data.utils import DBConnection
from config import DBConfig

In [24]:
class NotebookDBConnection(DBConnection):
    def __init__(self):
        db_path = os.path.join(project_root, 'data', 'pitcher_stats.db')
        super().__init__(db_name=db_path)

In [25]:
pd.set_option('display.max_columns', 150)

In [32]:
with NotebookDBConnection() as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    print([table[0] for table in tables])

['pitcher_mapping', 'statcast_pitchers', 'team_batting', 'statcast_batters', 'game_level_pitchers', 'game_level_batters', 'predictive_pitch_features', 'batter_predictive_features', 'combined_predictive_features', 'train_predictive_pitch_features', 'test_predictive_pitch_features', 'train_batter_predictive_features', 'test_batter_predictive_features', 'team_season_features', 'train_combined_features', 'test_combined_features']


In [27]:
with NotebookDBConnection() as conn:
    df_batters = pd.read_sql_query("SELECT * FROM statcast_batters LIMIT 25000;", conn)
    df_pitchers = pd.read_sql_query("SELECT * FROM statcast_pitchers LIMIT 25000;", conn)
    df_team_batting = pd.read_sql_query("SELECT * FROM team_batting;", conn)
    

In [28]:
with NotebookDBConnection() as conn:
    df_game_level = pd.read_sql_query("SELECT * FROM game_level_pitchers LIMIT 10000;", conn)
    df_pred = pd.read_sql_query("SELECT * FROM predictive_pitch_features LIMIT 10000;", conn)

In [33]:
with NotebookDBConnection() as conn:
    cursor = conn.cursor()
    cursor.execute('''SELECT pitcher_id, game_pk, game_date, COUNT(*)
        FROM game_level_pitchers
        GROUP BY pitcher_id, game_pk, game_date
        HAVING COUNT(*) > 1;''')
    print(cursor.fetchall())

[]


In [13]:
df_game_level.to_csv('game_level_pitchers.csv')

In [14]:
df_pred.to_csv('predictive_pitch_features.csv')

In [22]:
df_pitchers.sort_values(by="game_date").tail()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,fielder_2,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length,estimated_slg_using_speedangle,delta_pitcher_run_exp,hyper_speed,home_score_diff,bat_score_diff,home_win_exp,bat_win_exp,age_pit_legacy,age_bat_legacy,age_pit,age_bat,n_thruorder_pitcher,n_priorpa_thisgame_player_at_bat,pitcher_days_since_prev_game,batter_days_since_prev_game,pitcher_days_until_next_game,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle,pitcher_id,season
12193,FC,2025-04-06,94.4,-0.75,6.10,"Burnes, Corbin",669743,669203,None,ball,None,None,None,None,14.0,Alex Call walks. Josh Bell to 2nd.,R,R,R,WSH,AZ,B,NaN,None,2,1,2025,0.30,0.97,2.01,1.40,NaN,NaN,605137.0,2,3,Bot,NaN,NaN,None,None,None,None,6.546529,-137.170018,-8.737907,2.406804,29.625059,-18.224947,3.54,1.69,NaN,NaN,NaN,95.0,2688.0,6.7,778429,645444,647304,641658,553993,672695,666971,677950,682998,53.84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29,4,Cutter,3,2,3,2,2,3,3,2,Standard,Standard,195.0,0.000,0.078,NaN,NaN,NaN,-0.078,NaN,1,1,0.650,0.650,30,30,31,31,2,1,5.0,1.0,NaN,NaN,1.59,-0.30,-0.30,42.9,669203,2025
12192,FC,2025-04-06,94.4,-0.77,6.17,"Burnes, Corbin",669743,669203,None,called_strike,None,None,None,None,5.0,Alex Call walks. Josh Bell to 2nd.,R,R,R,WSH,AZ,S,NaN,None,3,1,2025,0.33,0.98,0.11,2.73,NaN,NaN,605137.0,2,3,Bot,NaN,NaN,None,None,None,None,1.555764,-137.495965,-5.526066,3.854207,27.660734,-18.774620,3.52,1.68,NaN,NaN,NaN,95.2,2729.0,6.5,778429,645444,647304,641658,553993,672695,666971,677950,682998,54.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29,5,Cutter,3,2,3,2,2,3,3,2,Standard,Standard,199.0,0.000,-0.065,NaN,NaN,NaN,0.065,NaN,1,1,0.650,0.650,30,30,31,31,2,1,5.0,1.0,NaN,NaN,1.56,-0.33,-0.33,44.7,669203,2025
12191,FC,2025-04-06,94.3,-0.65,6.22,"Burnes, Corbin",669743,669203,walk,ball,None,None,None,None,14.0,Alex Call walks. Josh Bell to 2nd.,R,R,R,WSH,AZ,B,NaN,None,3,2,2025,0.38,0.91,1.43,2.00,NaN,NaN,605137.0,2,3,Bot,NaN,NaN,None,None,None,None,4.566706,-137.207850,-7.421722,3.978941,26.194283,-19.351877,3.44,1.69,NaN,NaN,NaN,95.4,2703.0,6.6,778429,645444,647304,641658,553993,672695,666971,677950,682998,53.89,NaN,0.698324,0.7,1.0,0.0,0.0,NaN,29,6,Cutter,3,2,3,2,2,3,3,2,Standard,Standard,187.0,0.018,0.217,NaN,NaN,NaN,-0.217,NaN,1,1,0.650,0.650,30,30,31,31,2,1,5.0,1.0,NaN,NaN,1.63,-0.38,-0.38,45.4,669203,2025
12166,SI,2025-04-06,95.4,-0.70,6.19,"Burnes, Corbin",657557,669203,strikeout,called_strike,None,None,None,None,7.0,Paul DeJong called out on strikes.,R,R,R,WSH,AZ,S,2.0,None,1,2,2025,-0.74,1.34,-0.59,1.94,NaN,NaN,669743.0,2,5,Bot,NaN,NaN,None,None,None,None,1.968546,-138.698410,-8.571953,-9.930258,33.643851,-13.073980,3.37,1.56,NaN,NaN,NaN,95.6,2637.0,6.7,778429,645444,647304,641658,553993,672695,666971,677950,682998,53.83,NaN,0.000000,0.0,1.0,0.0,0.0,NaN,48,5,Sinker,4,3,4,3,3,4,4,3,Standard,Standard,231.0,-0.022,-0.196,NaN,NaN,NaN,0.196,NaN,1,1,0.689,0.689,30,31,31,32,3,2,5.0,1.0,NaN,NaN,1.18,0.74,0.74,44.6,669203,2025
12178,FC,2025-04-06,93.7,-1.01,6.03,"Burnes, Corbin",663993,669203,None,b

In [24]:
df_team_batting.to_csv('team_batting.csv')
df_batters.to_csv('statcast_batters_subset.csv')
df_pitchers.to_csv('statcast_pitchers_subset.csv')